Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

C:\Users\ajenk\GitHub\DS-Unit-2-Applied-Modeling\module2-wrangle-ml-datasets\instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

aisles.csv
departments.csv
orders.csv
order_products__prior.csv
order_products__train.csv
products.csv


### For each csv file, look at its shape & head 

In [6]:
import pandas as pd

In [15]:
from IPython.display import display

def preview():
    for filename in glob('*.csv'):
        file = pd.read_csv(filename)
        print('\n', filename, file.shape)
        display(file.head())
    
preview()


 aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


Look at the IDs here -- this is how we're going to join all the tables together! To get to orders to aisles, we'd have to connect orders to order_products to products to aisles. For merging data,it doesn't hurt to use physical paper or a white board to keep track!

## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [18]:
prior = pd.read_csv("order_products__prior.csv")

In [19]:
prior["product_id"].value_counts()

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
          ...  
11356         1
18001         1
6320          1
26268         1
30087         1
Name: product_id, Length: 49677, dtype: int64

In [21]:
products = pd.read_csv("products.csv")

In [23]:
products[products["product_id"] == 24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [24]:
train = pd.read_csv('order_products__train.csv')
train['product_id'].mode()

0    24852
dtype: int64

In [25]:
train['order_id'].nunique()

131209

In [26]:
prior = pd.merge(prior, products, on = "product_id", how = "inner")
#assigning over prior data frame - be careful doing this, but 
# frees up memory compared to having a bunch lying around.

## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [27]:
train['banana'] = train["product_id"] == 24852

In [29]:
train["banana"].sum()

18726

In [34]:
train.groupby('order_id')["banana"].any()

order_id
1          False
36         False
38         False
96         False
98         False
           ...  
3421049    False
3421056    False
3421058    False
3421063    False
3421070    False
Name: banana, Length: 131209, dtype: bool

In [36]:
train_wrangled = train.groupby('order_id')["banana"].any().reset_index()

In [39]:
train_wrangled["banana"].mean()

0.14271886837030998

## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [40]:
bananas = 24852
banana_prior_order_ids = prior[prior.product_id==bananas]["order_id"]

In [41]:
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3189197,2972574,192282,prior,4,5,12,11.0
1342289,986657,80645,prior,11,0,15,6.0
1224142,173638,73571,prior,5,4,15,3.0
1433190,1901040,86133,prior,32,1,8,8.0
1024316,1248656,61576,prior,3,4,18,6.0


In [43]:
banana_orders = orders[orders["order_id"].isin(banana_prior_order_ids)]

In [44]:
banana_user_ids = banana_orders["user_id"].unique()

In [45]:
banana_user_ids[:5]

array([ 2, 10, 16, 21, 27], dtype=int64)

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [46]:
orders = orders[orders['user_id'].isin(banana_user_ids)]
orders.shape

(1512975, 7)

In [47]:
subset_order_ids = orders.order_id.unique()

In [48]:
prior.shape

(32434489, 7)

In [49]:
prior = prior[prior.order_id.isin(subset_order_ids)]
prior.shape

(16534534, 7)

In [50]:
train.shape

(1384617, 5)

In [51]:
train = train[train.order_id.isin(subset_order_ids)]
train.shape

(587269, 5)

In [53]:
target = 'banana'
train[target].value_counts(normalize = True)

False    0.971807
True     0.028193
Name: banana, dtype: float64

In [54]:
train.head(100)

,order_id,product_id,add_to_cart_order,reordered,banana
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False
...,...,...,...,...,...
270,1032,9133,24,0,False
271,1032,35352,25,0,False
272,1042,24852,1,1,True
273,1042,33000,2,1,False


## What features can we engineer? We want to predict, will these customers reorder this product on their next order?

In [55]:
train = pd.merge(train, orders)